In [1]:
rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr)

Cargando paquete requerido: pacman



In [2]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

In [3]:
url_basica<-"https://www.inegi.org.mx/contenidos/programas/enigh/nc/"
segunda_parte<-"microdatos/enigh"
fin1<-"_ns_"
fin2<-"_csv.zip"

In [4]:
years<-c("2022")

In [5]:
# Función genérica para descargar, descomprimir, renombrar y eliminar
procesar_archivo <- function(year, tipo, url_base, segunda_parte, fin1, fin2, carpeta_destino) {
  # Construir URL y ruta de destino
  url <- paste0(url_base, year, "/", segunda_parte, year, fin1, tipo, fin2)
  destfile <- paste0(carpeta_destino, "/", year, fin1, tipo, fin2)

  # Descargar el archivo
  download.file(url, destfile = destfile)

  # Descomprimir el archivo
  unzip(destfile, exdir = carpeta_destino)

  # Renombrar el archivo descomprimido
  file.rename(
    from = paste0(carpeta_destino, "/", tipo, ".csv"),
    to = paste0(carpeta_destino, "/", tipo, "_", year, ".csv")
  )

  # Eliminar el archivo descargado
  file.remove(destfile)
}

In [6]:
# Iterar sobre los años y procesar cada tipo de archivo

for (i in 1:length(years)) {
  procesar_archivo(years[i], "poblacion", url_basica, segunda_parte, fin1, fin2, "data")
  procesar_archivo(years[i], "concentradohogar", url_basica, segunda_parte, fin1, fin2, "data")
}

In [ ]:
cuidadores<-read_csv("data/poblacion_2022.csv", show_col_types = FALSE)%>%
  janitor::clean_names()%>%
  #Se crea clave de entidad
  mutate(cve_ent = case_when(
    nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
    nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
  ))%>%
  #Filtrar para Ciudad de México
  filter(cve_ent == "09")%>%

  #Transformar tiempos de cuidados a días para poder calcular ingresos
  mutate(
  #Transformar minutos a horas
  horas_cuidados=min_4/60,
  tiempo_cuidados=horas_cuidados+hor_4,
  #Pasar horas a días considerando que son horas semanales
    cuidados_semanales_dias=tiempo_cuidados/24,
    #Identificar si la persona es cuidadora
cuidador = ifelse(tiempo_cuidados > 0 | !is.na(tiempo_cuidados), 1, 0)
    )%>%
    #Filtrar cuidadores
    
    filter(parentesco==101)%>%

    #Seleccionar variables de interés
    select(folioviv,foliohog,cuidador,tiempo_cuidados,hor_4,min_4,cuidados_semanales_dias,factor,est_dis,upm)


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


folioviv,foliohog,cuidador,tiempo_cuidados,hor_4,min_4,cuidados_semanales_dias,factor,est_dis,upm
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
0960055618,1,NA,NA,NA,NA,NA,43,145,0003496
0960055901,1,NA,NA,NA,NA,NA,43,145,0003496
0960055902,1,NA,NA,NA,NA,NA,43,145,0003496
0960055903,1,NA,NA,NA,NA,NA,43,145,0003496
0960055904,1,NA,NA,NA,NA,NA,43,145,0003496
0960055905,1,NA,NA,NA,NA,NA,43,145,0003496


In [ ]:
#Diseño muestral
dm<-cuidadores%>%
as_survey_design(ids = upm,
                     strata = est_dis,
                     weights = factor)

In [ ]:
#Total hogares en la Ciudad de México
dm%>%
    summarise(total_hogares = survey_total(vartype = "cv"))

total_hogares,total_hogares_cv
<dbl>,<dbl>
2990030,0.01388237


In [ ]:
#Hogares cuidadores por semejanza a la jornada laboral
dm%>%
filter(cuidador==1)%>%
    mutate(cuidado_jornadalab=case_when(
    tiempo_cuidados>=6*8 ~ 1,
    TRUE ~ 0
    ))%>%
  group_by(cuidado_jornadalab)%>%
  summarise(cuidadores=survey_total(vartype = "cv"),
            cuidadores_prop=survey_mean(vartype = "cv"))%>%
            janitor::adorn_totals()

cuidado_jornadalab,cuidadores,cuidadores_cv,cuidadores_prop,cuidadores_prop_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
0,460118,0.06130674,0.8944038,0.01966505
1,54323,0.17752347,0.1055962,0.16656375
Total,514441,0.23883021,1.0000000,0.18622880


: 